In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
# Đường dẫn đến tập tin zip trên Google Drive
zip_path = '/content/drive/MyDrive/YOLOv8/brain-tumour-dataset.zip'
# Sao chép tập tin zip vào thư mục làm việc của Colab
!cp "{zip_path}" .
# Giải nén tập tin zip
!unzip -q brain-tumour-dataset.zip

In [5]:
import os
import shutil
import glob
import pandas as pd
import cv2
import numpy as np
import random

In [6]:
 # Đường dẫn chứa ảnh và mặt nạ
path = '/content/brain-tumour-dataset/lgg-mri-segmentation/kaggle_3m/'
# Phân chia ảnh và các mặt nạ (ảnh đen trắng) tương ứng trong đường dẫn
masks_dir = glob.glob(path + "*/*_mask*")
images_dir = []
# Lặp qua từng đường dẫn trong mask_dir
for img in masks_dir:
    image_path = img.replace("_mask", "")
    images_dir.append(image_path)
    #print("Image path:", image_path)  # Kiểm tra danh sách images_dir

# Tạo ra dataframe mới cho ảnh não bộ và ảnh mặt nạ tương ứng
#tạo ra các dataframe mới cho ảnh não bộ và ảnh mặt nạ tương ứng
data_brain = pd.DataFrame(data={
                                "file_images":images_dir,
                                "file_masks":masks_dir
                                })
data_brain.head()

,file_images,file_masks
0,/content/brain-tumour-dataset/lgg-mri-segmenta...,/content/brain-tumour-dataset/lgg-mri-segmenta...
1,/content/brain-tumour-dataset/lgg-mri-segmenta...,/content/brain-tumour-dataset/lgg-mri-segmenta...
2,/content/brain-tumour-dataset/lgg-mri-segmenta...,/content/brain-tumour-dataset/lgg-mri-segmenta...
3,/content/brain-tumour-dataset/lgg-mri-segmenta...,/content/brain-tumour-dataset/lgg-mri-segmenta...
4,/content/brain-tumour-dataset/lgg-mri-segmenta...,/content/brain-tumour-dataset/lgg-mri-segmenta...


In [7]:
def positive_negative_diagnosis(file_masks):
    mask = cv2.imread(file_masks)
    value = np.max(mask)
    if value > 0 :
        return 1
    else :
        return 0


In [8]:
data_brain['mask'] = data_brain['file_masks'].apply(lambda x: positive_negative_diagnosis(x))
data_brain

,file_images,file_masks,mask
0,/content/brain-tumour-dataset/lgg-mri-segmenta...,/content/brain-tumour-dataset/lgg-mri-segmenta...,1
1,/content/brain-tumour-dataset/lgg-mri-segmenta...,/content/brain-tumour-dataset/lgg-mri-segmenta...,0
2,/content/brain-tumour-dataset/lgg-mri-segmenta...,/content/brain-tumour-dataset/lgg-mri-segmenta...,1
3,/content/brain-tumour-dataset/lgg-mri-segmenta...,/content/brain-tumour-dataset/lgg-mri-segmenta...,1
4,/content/brain-tumour-dataset/lgg-mri-segmenta...,/content/brain-tumour-dataset/lgg-mri-segmenta...,1
...,...,...,...
3924,/content/brain-tumour-dataset/lgg-mri-segmenta...,/content/brain-tumour-dataset/lgg-mri-segmenta...,1
3925,/content/brain-tumour-dataset/lgg-mri-segmenta...,/content/brain-tumour-dataset/lgg-mri-segmenta...,1
3926,/content/brain-tumour-dataset/lgg-mri-segmenta...,/content/brain-tumour-dataset/lgg-mri-segmenta...,1
3927,/content/brain-tumour-dataset/lgg-mri-segmenta...,/content/brain-tumour-dataset/lgg-mri-segmenta...,1


In [9]:
data_brain['mask'].value_counts()

,count
mask,
0,2556
1,1373


In [10]:
def positive_negative_data(file_mask , output_mask, output_image):
    mask = cv2.imread(file_mask)
    value = np.max(mask)
    if value > 0:
        #tạo tên file tương ứng
        mask_name = os.path.basename(file_mask)
        image_name = mask_name.replace("_mask","")
        #đọc ảnh gốc tương ứng
        image_path = file_mask.replace("_mask","")
        image = cv2.imread(image_path)
        #lưu ảnh mask vào output_mask
        mask_dest = os.path.join(output_mask,mask_name)
        cv2.imwrite(mask_dest,mask)
        #lưu ảnh gốc vào output_image
        image_dest = os.path.join(output_image,image_name)
        cv2.imwrite(image_dest, image)
        print(f'Đã lưu ảnh :{image_name} và mask:{mask_name}')
    else:
        print(f'không có ảnh có khối u trong {file_mask}')
mask_output_dir = "/content/drive/MyDrive/YOLOv8/masks"
image_output_dir = "/content/drive/MyDrive/YOLOv8/images"
# Lặp qua từng ảnh mặt nạ và kiểm tra, lưu ảnh
for file_masks in masks_dir:
    positive_negative_data(file_masks, mask_output_dir, image_output_dir)


Đã lưu ảnh :TCGA_HT_7473_19970826_12.tif và mask:TCGA_HT_7473_19970826_12_mask.tif
không có ảnh có khối u trong /content/brain-tumour-dataset/lgg-mri-segmentation/kaggle_3m/TCGA_HT_7473_19970826/TCGA_HT_7473_19970826_4_mask.tif
Đã lưu ảnh :TCGA_HT_7473_19970826_16.tif và mask:TCGA_HT_7473_19970826_16_mask.tif
Đã lưu ảnh :TCGA_HT_7473_19970826_22.tif và mask:TCGA_HT_7473_19970826_22_mask.tif
Đã lưu ảnh :TCGA_HT_7473_19970826_10.tif và mask:TCGA_HT_7473_19970826_10_mask.tif
không có ảnh có khối u trong /content/brain-tumour-dataset/lgg-mri-segmentation/kaggle_3m/TCGA_HT_7473_19970826/TCGA_HT_7473_19970826_30_mask.tif
không có ảnh có khối u trong /content/brain-tumour-dataset/lgg-mri-segmentation/kaggle_3m/TCGA_HT_7473_19970826/TCGA_HT_7473_19970826_9_mask.tif
không có ảnh có khối u trong /content/brain-tumour-dataset/lgg-mri-segmentation/kaggle_3m/TCGA_HT_7473_19970826/TCGA_HT_7473_19970826_2_mask.tif
Đã lưu ảnh :TCGA_HT_7473_19970826_13.tif và mask:TCGA_HT_7473_19970826_13_mask.tif
khôn

In [11]:
print("ảnh mask:",len(os.listdir(mask_output_dir)))
print("ảnh gốc:",len(os.listdir(image_output_dir)))

ảnh mask: 1373
ảnh gốc: 1373


In [15]:
#Đường dẫn tới thư mục chứa các ảnh .tif
image_dir = "/content/drive/MyDrive/YOLOv8/dataset/images_and_labels/masks/val"
# Lấy tất cả các tệp .tif trong thư mục
image_files = [f for f in os.listdir(image_dir) if f.lower().endswith('.tif')]
# Sắp xếp danh sách ảnh (nếu cần)
image_files.sort()
# Lặp qua các file và đổi tên
for i, img in enumerate(image_files, start=1):
    # Tạo tên mới cho ảnh
    new_name = f"val{i}.tif"
    # Đường dẫn cũ và đường dẫn mới
    old_path = os.path.join(image_dir, img)
    new_path = os.path.join(image_dir, new_name)
    # Đổi tên ảnh
    os.rename(old_path, new_path)
    print(f"Đã đổi tên {img} thành {new_name}")
print("Đổi tên tất cả các ảnh hoàn tất.")


Đã đổi tên TCGA_CS_4941_19960909_11_mask.tif thành val1.tif
Đã đổi tên TCGA_CS_4941_19960909_12_mask.tif thành val2.tif
Đã đổi tên TCGA_CS_4941_19960909_13_mask.tif thành val3.tif
Đã đổi tên TCGA_CS_4941_19960909_14_mask.tif thành val4.tif
Đã đổi tên TCGA_CS_4941_19960909_15_mask.tif thành val5.tif
Đã đổi tên TCGA_CS_4941_19960909_16_mask.tif thành val6.tif
Đã đổi tên TCGA_CS_4941_19960909_17_mask.tif thành val7.tif
Đã đổi tên TCGA_CS_4941_19960909_18_mask.tif thành val8.tif
Đã đổi tên TCGA_CS_4942_19970222_10_mask.tif thành val9.tif
Đã đổi tên TCGA_CS_4942_19970222_11_mask.tif thành val10.tif
Đã đổi tên TCGA_CS_4942_19970222_12_mask.tif thành val11.tif
Đã đổi tên TCGA_CS_4942_19970222_13_mask.tif thành val12.tif
Đã đổi tên TCGA_CS_4942_19970222_14_mask.tif thành val13.tif
Đã đổi tên TCGA_CS_4942_19970222_9_mask.tif thành val14.tif
Đã đổi tên TCGA_CS_4943_20000902_12_mask.tif thành val15.tif
Đã đổi tên TCGA_CS_4943_20000902_13_mask.tif thành val16.tif
Đã đổi tên TCGA_CS_4943_20000902_1

In [17]:
def mask_to_yolo_polygon(mask_path, output_txt_path, class_id=0):
    # Đọc ảnh mask
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    # Lấy kích thước ảnh
    h, w = mask.shape
    # Tìm contours (đường bao)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    with open(output_txt_path, 'w') as f:
        for contour in contours:
            # Lấy các điểm contour
            points = contour.squeeze(axis=1)  # (n, 1, 2) -> (n, 2)
            # Chuẩn hóa tọa độ (0-1)
            normalized_points = [(x / w, y / h) for x, y in points]
            # Tạo chuỗi YOLO Polygon
            yolo_polygon = [class_id] + [coord for point in normalized_points for coord in point]
            yolo_polygon_str = " ".join(map(str, yolo_polygon))
            # Ghi vào tệp .txt
            f.write(yolo_polygon_str + '\n')
    print(f"YOLO Polygon file saved to: {output_txt_path}")
def convert_mask_folder_to_txt(mask_folder, output_folder, class_id=0):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for mask_file in os.listdir(mask_folder):
        if mask_file.endswith(('.png', '.jpg', '.tif')):
            mask_path = os.path.join(mask_folder, mask_file)
            output_txt_path = os.path.join(output_folder, os.path.splitext(mask_file)[0] + ".txt")
            mask_to_yolo_polygon(mask_path, output_txt_path, class_id)
mask_folder = "/content/drive/MyDrive/YOLOv8/dataset/images_and_labels/masks/val"  # Thư mục chứa ảnh mask
output_folder = "/content/drive/MyDrive/YOLOv8/dataset/images_and_labels/labels/val"  # Thư mục lưu nhãn YOLO Polygon

convert_mask_folder_to_txt(mask_folder, output_folder, class_id=0)


YOLO Polygon file saved to: /content/drive/MyDrive/YOLOv8/dataset/images_and_labels/labels/val/val11.txt
YOLO Polygon file saved to: /content/drive/MyDrive/YOLOv8/dataset/images_and_labels/labels/val/val12.txt
YOLO Polygon file saved to: /content/drive/MyDrive/YOLOv8/dataset/images_and_labels/labels/val/val13.txt
YOLO Polygon file saved to: /content/drive/MyDrive/YOLOv8/dataset/images_and_labels/labels/val/val14.txt
YOLO Polygon file saved to: /content/drive/MyDrive/YOLOv8/dataset/images_and_labels/labels/val/val9.txt
YOLO Polygon file saved to: /content/drive/MyDrive/YOLOv8/dataset/images_and_labels/labels/val/val10.txt
YOLO Polygon file saved to: /content/drive/MyDrive/YOLOv8/dataset/images_and_labels/labels/val/val162.txt
YOLO Polygon file saved to: /content/drive/MyDrive/YOLOv8/dataset/images_and_labels/labels/val/val160.txt
YOLO Polygon file saved to: /content/drive/MyDrive/YOLOv8/dataset/images_and_labels/labels/val/val156.txt
YOLO Polygon file saved to: /content/drive/MyDrive/YO

In [19]:
cfg = '/content/drive/MyDrive/YOLOv8/YOLOv8-Image-Segmentation-main/yolov8x-seg_custom.yaml'

In [30]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics 8.3.39 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 36.5/107.7 GB disk)


In [31]:
from ultralytics import YOLO
#huấn luyện
!yolo task=segment mode=train model= /content/drive/MyDrive/YOLOv8/yolov8n-seg.pt data= /content/drive/MyDrive/YOLOv8/YOLOv8-Image-Segmentation-main/custom_data.yaml epochs=10 batch= 16 project= '/content/drive/MyDrive/YOLOv8/results_run' name='segment_train'


Ultralytics 8.3.39 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=segment, mode=train, model=/content/drive/MyDrive/YOLOv8/yolov8n-seg.pt, data=/content/drive/MyDrive/YOLOv8/YOLOv8-Image-Segmentation-main/custom_data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/YOLOv8/results_run, name=segment_train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=No

In [32]:
#test
!yolo task=segment mode=predict model=/content/drive/MyDrive/YOLOv8/results_run/segment_train2/weights/best.pt source='/content/drive/MyDrive/YOLOv8/test' save = True project='/content/drive/MyDrive/YOLOv8/result_test' name='yolo_test_result'

Ultralytics 8.3.39 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
YOLOv8n-seg summary (fused): 195 layers, 3,258,259 parameters, 0 gradients, 12.0 GFLOPs

image 1/11 /content/drive/MyDrive/YOLOv8/test/test1.jpg: 640x640 1 Tumor, 371.8ms
image 2/11 /content/drive/MyDrive/YOLOv8/test/test10.jpg: 640x640 1 Tumor, 318.4ms
image 3/11 /content/drive/MyDrive/YOLOv8/test/test11.jpg: 640x640 1 Tumor, 331.9ms
image 4/11 /content/drive/MyDrive/YOLOv8/test/test2.jpg: 640x640 1 Tumor, 303.8ms
image 5/11 /content/drive/MyDrive/YOLOv8/test/test3.jpg: 640x640 1 Tumor, 300.1ms
image 6/11 /content/drive/MyDrive/YOLOv8/test/test4.jpg: 640x640 1 Tumor, 460.2ms
image 7/11 /content/drive/MyDrive/YOLOv8/test/test5.jpg: 640x640 1 Tumor, 490.6ms
image 8/11 /content/drive/MyDrive/YOLOv8/test/test6.jpg: 640x640 1 Tumor, 477.4ms
image 9/11 /content/drive/MyDrive/YOLOv8/test/test7.jpg: 640x640 1 Tumor, 530.7ms
image 10/11 /content/drive/MyDrive/YOLOv8/test/test8.jpg: 640x640 1 Tumor, 321.8ms
image 11/